<a href="https://colab.research.google.com/github/sagiri262/DataMining-2025Fall-MUST/blob/main/Assignment2_2250016066.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2:
# Mining Frequent Patterns

In this assignment, we will using Apriori algorithm to mine the frequent patterns (sort terms, just set k = 3, 4, 5) from a social network dataset.

##Data Preparation

At first, download the dataset from http://snap.stanford.edu/data/gemsec_facebook_dataset.tar.gz and read the dataset description. Then upload and connect it to your VM.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In this pratice, we just use the first datafile (artist_edges.csv).

Now load the data to a variable in python.

In [ ]:
labdir = "/content/sample_data/facebook_clean_data/"
import csv

with open(labdir+"tvshow_edges.csv") as csvfile:
    facebook_clean_data = list(csv.reader(csvfile))

##Apriori Algorithm

Apriori is an algorithm for frequent item set mining and association rule learning over relational databases.

Before to perform Apriori, we need to define a function 'data_processing' for data preprocessing firstly.

'data_set' is a list of transactions. Each transaction contains several items.

In [ ]:
def processing_data():
  dic = {}
  for i in facebook_clean_data[1:]:
    if not i[0] in dic:
      dic[i[0]] = [i[1]]
    else:
      dic[i[0]].append(i[1])
  data_set = list(dic.values())
  return data_set
data_set = processing_data()

Now define a function 'create_C1' ot create frequent candidate 1-itemset C1 by scaning data set.

C1: A set which contains all frequent candidate 1-itemsets.

In [ ]:
def create_C1(data_set):
    C1 = set()
    for t in data_set:
        for item in t:
            item_set = frozenset([item])
            C1.add(item_set)
    return C1

Judge whether a frequent candidate k-itemset satisfy Apriori property.

Ck_item: A frequent candidate k-itemset in Ck which contains all frequent candidate k-itemsets.

Lksub1: A set which contains all frequent candidate (k-1)-itemsets.

In [ ]:
def is_apriori(Ck_item, Lksub1):
    for item in Ck_item:
        sub_Ck = Ck_item - frozenset([item])
        if sub_Ck not in Lksub1:
            return False
    return True

Create Ck, a set which contains all all frequent candidate k-itemsets
    by Lk-1's own connection operation.

k: the item number of a frequent itemset.

Ck: A set which contains all all frequent candidate k-itemsets.

In [ ]:
def create_Ck(Lksub1, k):
# ToDo: Write you code here.
    Ck = set()
    list_Lksub1 = list(Lksub1)
    len_Lksub1 = len(list_Lksub1)
    for i in range(len_Lksub1):
        for j in range(i+1, len_Lksub1):
            L1 = list(list_Lksub1[i])[:k-2]
            L2 = list(list_Lksub1[j])[:k-2]
            L1.sort()
            L2.sort()

            # 仅当 (k-2) 个项相同时才进行连接
            if L1 == L2:
                # 连接操作：L1和L2的并集
                Ck_item = list_Lksub1[i] | list_Lksub1[j]
                # 剪枝
                if is_apriori(Ck_item, Lksub1):
                    Ck.add(Ck_item)
    return Ck

Generate Lk by executing a delete policy from Ck.

min_support: The minimum support.

support_data: A dictionary. The key is frequent itemset and the value is support.

Lk: A set which contains all all frequent k-itemsets.

In [ ]:
def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
# ToDo: Write you code here.
    item_count = {}
    Lk = set()
    num_transactions = len(data_set)

    # 遍历数据集计算 Ck 中每个候选项集的支持度计数
    for t in data_set:
        transcation = frozenset(t)
        for item in Ck:
            if item.issubset(transcation):
                if item not in item_count:
                    item_count[item] = item_count.get(item, 0) + 1

    # 遍历候选项集，保留支持度大于等于 min_support 的项集，生成 Lk
    for item, count in item_count.items():
        support = count / num_transactions
        if support >= min_support:
            Lk.add(item)
            support_data[item] = support

    return Lk

Generate all frequent itemsets.

k: Maximum number of items for all frequent itemsets.

L: The list of Lk.

In [ ]:
def generate_L(data_set, k, min_support):
# ToDo: Write you code here.
    support_data = {}
    C1 = create_C1(data_set)
    L1 = generate_Lk_by_Ck(data_set, C1, min_support, support_data)
    L = [L1]

    # 从 k = 2 开始循环，直到 Lk 为空或达到最大项数 k
    for i in range(2, k+2):
        Lksub1 = L[i-2]
        Ck = create_Ck(Lksub1, i)

        Lk = generate_Lk_by_Ck(data_set, Ck, min_support, support_data)

        # 如果 Lk 为空，则停止循环
        if not Lk:
            break

        L.append(Lk)
    return L, support_data

Generate big rules from frequent itemsets.

min_conf: Minimal confidence.

big_rule_list: A list which contains all big rules. Each big rule is represented as a 3-tuple.

In [ ]:
def generate_big_rules(L, support_data, min_conf):
# ToDo: Write you code here.
    big_rule_list = []

    for i in range(1, len(L)):
        for freq_set in L[i]:
            H1 = [frozenset([item]) for item in freq_set]
            if i > 1:
                rules_from_conseq(freq_set, H1, support_data, big_rule_list, min_conf)
            else:
                calc_conf(freq_set, H1, support_data, big_rule_list, min_conf)
    return big_rule_list


def calc_conf(freq_set, H, support_data, big_rule_list, min_conf):
    pruned_H = []
    for conseq in H:
        # 计算置信度
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_conf:
            # 添加规则
            big_rule_list.append((freq_set - conseq, conseq, conf))
            pruned_H.append(conseq)
    return pruned_H


# 辅助函数：从 k-项集生成规则的后件
def rules_from_conseq(freq_set, H, support_data, big_rule_list, min_conf):
    m = len(H[0])
    if len(freq_set) > (m + 1):
        # H 是 m 项集后件的列表
        # 从 m-项集后件生成 (m+1)-项集后件
        Hmp1 = create_Ck(H, m + 1)
        Hmp1 = calc_conf(freq_set, Hmp1, support_data, big_rule_list, min_conf)

        # 如果还有可以进一步生成的后件，则递归调用
        if len(Hmp1) > 1:
            rules_from_conseq(freq_set, Hmp1, support_data, big_rule_list, min_conf)


In [ ]:
if __name__ == '__main__':
    # 调用 processing_data 获取数据集
    # 注意：这里使用了一个示例数据集，如果运行您的实际数据，请确保 facebook_clean_data 变量已定义
    data_set = processing_data()

    # 示例数据
    # data_set = [['l1', 'l2', 'l5'], ['l2', 'l4'], ['l2', 'l3'], ['l1', 'l2', 'l4'], ['l1', 'l3'], ['l2', 'l3'],
    #            ['l1', 'l3'], ['l1', 'l2', 'l3', 'l5'], ['l1', 'l2', 'l3']]

    # 生成所有频繁项集
    # k=3: 期望的最大频繁项集项数
    # min_support=0.2: 最小支持度阈值
    L, support_data = generate_L(data_set, k=3, min_support=0.2)

    # 生成关联规则
    # min_conf=0.7: 最小置信度阈值
    big_rules_list = generate_big_rules(L, support_data, min_conf=0.7)

    # 打印结果
    print(f"生成的频繁项集列表 L 的长度: {len(L)}")
    print(f"L[0] (L1) 中的项集数量: {len(L[0])}")

    for Lk in L:
        print("="*50)
        # 获取项集的大小
        if Lk:
            k_size = len(list(Lk)[0])
            print(f"频繁 {k_size}-项集\t\t支持度")
        else:
            print("空集")
        print("="*50)
        for freq_set in Lk:
            # support_data[freq_set] 存储着支持度
            print(freq_set, round(support_data[freq_set], 3))

    print("\n大规则 (关联规则)")
    for item in big_rules_list:
        # item: (前件, 后件, 置信度)
        print(f"{set(item[0])} => {set(item[1])} \t 置信度: {round(item[2], 3)}")

生成的频繁项集列表 L 的长度: 1
L[0] (L1) 中的项集数量: 0
空集

大规则 (关联规则)


In [ ]:
data_set = processing_data()
# data_set = [['l1', 'l2', 'l5'], ['l2', 'l4'], ['l2', 'l3'], ['l1', 'l2', 'l4'], ['l1', 'l3'], ['l2', 'l3'],
#            ['l1', 'l3'], ['l1', 'l2', 'l3', 'l5'], ['l1', 'l2', 'l3']]
L, support_data = generate_L(data_set, k=3, min_support=0.2)
big_rules_list = generate_big_rules(L, support_data, min_conf=0.7)
print(len(L))
print(L[0])
for Lk in L:
  print("="*50)
  print("frequent " + str(len(list(Lk)[0])) + "-itemsets\t\tsupport")
  print("="*50)
  for freq_set in Lk:
    print(freq_set, support_data[freq_set])
print("Big Rules")
for item in big_rules_list:
  print(item[0], "=>", item[1], "conf: ", item[2])

We can download data in this way:
open the ternimal and enter "wget http://snap.stanford.edu/data/gemsec_facebook_dataset.tar.gz" download data to ./sample_data. Then use command "tar -xvf gemsec_facebook_dataset.tar.gz" unzip the data to folder.

In [3]:
 # Apriori Alogrithm Codes

import csv
import itertools

labdir = "/content/sample_data/"

with open(labdir+"tvshow_edges.csv") as csvfile:
    facebook_clean_data = list(csv.reader(csvfile))
    # print(facebook_clean_data[:5])

def processing_data():
    dic = {}
    for i in facebook_clean_data[1:]:
        if not i[0] in dic:
            dic[i[0]] = [i[1]]
        else:
            dic[i[0]].append(i[1])
    data_set = list(dic.values())
    return data_set
data_set = processing_data()


def create_C1(data_set):
    C1 = set()
    for t in data_set:
        for item in t:
            item_set = frozenset([item])
            C1.add(item_set)
    return C1


def is_apriori(Ck_item, Lksub1):
    for item in Ck_item:
        sub_Ck = Ck_item - frozenset([item])
        if sub_Ck not in Lksub1:
            return False
    return True


def create_Ck(Lksub1, k):
# ToDo: Write you code here.
    Ck = set()
    list_Lksub1 = list(Lksub1)
    len_Lksub1 = len(list_Lksub1)
    for i in range(len_Lksub1):
        for j in range(i+1, len_Lksub1):
            L1 = list(list_Lksub1[i])[:k-2]
            L2 = list(list_Lksub1[j])[:k-2]
            L1.sort()
            L2.sort()

            # 仅当 (k-2) 个项相同时才进行连接
            if L1 == L2:
                # 连接操作：L1和L2的并集
                Ck_item = list_Lksub1[i] | list_Lksub1[j]
                # 剪枝
                if is_apriori(Ck_item, Lksub1):
                    Ck.add(Ck_item)
    return Ck


def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
# ToDo: Write you code here.
    item_count = {}
    Lk = set()
    num_transactions = len(data_set)

    # 遍历数据集计算 Ck 中每个候选项集的支持度计数
    for t in data_set:
        transcation = frozenset(t)
        for item in Ck:
            if item.issubset(transcation):
                if item not in item_count:
                    item_count[item] = item_count.get(item, 0) + 1

    # 遍历候选项集，保留支持度大于等于 min_support 的项集，生成 Lk
    for item, count in item_count.items():
        support = count / num_transactions
        if support >= min_support:
            Lk.add(item)
            support_data[item] = support

    return Lk


def generate_L(data_set, k, min_support):
# ToDo: Write you code here.
    support_data = {}
    C1 = create_C1(data_set)
    L1 = generate_Lk_by_Ck(data_set, C1, min_support, support_data)
    L = [L1]

    # 从 k = 2 开始循环，直到 Lk 为空或达到最大项数 k
    for i in range(2, k+2):
        Lksub1 = L[i-2]
        Ck = create_Ck(Lksub1, i)

        Lk = generate_Lk_by_Ck(data_set, Ck, min_support, support_data)

        # 如果 Lk 为空，则停止循环
        if not Lk:
            break

        L.append(Lk)
    return L, support_data


def generate_big_rules(L, support_data, min_conf):
# ToDo: Write you code here.
    big_rule_list = []

    for i in range(1, len(L)):
        for freq_set in L[i]:
            H1 = [frozenset([item]) for item in freq_set]
            if i > 1:
                rules_from_conseq(freq_set, H1, support_data, big_rule_list, min_conf)
            else:
                calc_conf(freq_set, H1, support_data, big_rule_list, min_conf)
    return big_rule_list


def calc_conf(freq_set, H, support_data, big_rule_list, min_conf):
    pruned_H = []
    for conseq in H:
        # 计算置信度
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_conf:
            # 添加规则
            big_rule_list.append((freq_set - conseq, conseq, conf))
            pruned_H.append(conseq)
    return pruned_H


# 辅助函数：从 k-项集生成规则的后件
def rules_from_conseq(freq_set, H, support_data, big_rule_list, min_conf):
    m = len(H[0])
    if len(freq_set) > (m + 1):
        # H 是 m 项集后件的列表
        # 从 m-项集后件生成 (m+1)-项集后件
        Hmp1 = create_Ck(H, m + 1)
        Hmp1 = calc_conf(freq_set, Hmp1, support_data, big_rule_list, min_conf)

        # 如果还有可以进一步生成的后件，则递归调用
        if len(Hmp1) > 1:
            rules_from_conseq(freq_set, Hmp1, support_data, big_rule_list, min_conf)


if __name__ == '__main__':
    # 调用 processing_data 获取数据集
    # 注意：这里使用了一个示例数据集，如果运行您的实际数据，请确保 facebook_clean_data 变量已定义
    data_set = processing_data()

    # 示例数据
    # data_set = [['l1', 'l2', 'l5'], ['l2', 'l4'], ['l2', 'l3'], ['l1', 'l2', 'l4'], ['l1', 'l3'], ['l2', 'l3'],
    #            ['l1', 'l3'], ['l1', 'l2', 'l3', 'l5'], ['l1', 'l2', 'l3']]

    # 生成所有频繁项集
    # k=3: 期望的最大频繁项集项数
    # min_support=0.2: 最小支持度阈值
    L, support_data = generate_L(data_set, k=3, min_support=0.2)

    # 生成关联规则
    # min_conf=0.7: 最小置信度阈值
    big_rules_list = generate_big_rules(L, support_data, min_conf=0.7)

    # 打印结果
    print(f"生成的频繁项集列表 L 的长度: {len(L)}")
    print(f"L[0] (L1) 中的项集数量: {len(L[0])}")

    for Lk in L:
        print("="*50)
        # 获取项集的大小
        if Lk:
            k_size = len(list(Lk)[0])
            print(f"频繁 {k_size}-项集\t\t支持度")
        else:
            print("空集")
        print("="*50)
        for freq_set in Lk:
            # support_data[freq_set] 存储着支持度
            print(freq_set, round(support_data[freq_set], 3))

    print("\n大规则 (关联规则)")
    for item in big_rules_list:
        # item: (前件, 后件, 置信度)
        print(f"{set(item[0])} => {set(item[1])} \t 置信度: {round(item[2], 3)}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/tvshow_edges.csv'